In [1]:
suppressMessages(library('tidyverse'))

## Load datasets

More information on the pre-processing of each city's police misconduct complaints data in the [`cleaning`](https://github.com/Financial-Times/police-misconduct-complaints-analysis/tree/main/cleaning) directory.

In [2]:
chicago <- read_csv('output/chicago_clean.csv')
nyc <- read_csv('output/nyc_clean.csv')
philly <- read_csv('output/philly_clean.csv')


── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────
cols(
  .default = col_character(),
  cr_id = col_double(),
  link_UID = col_double(),
  UID = col_double(),
  cv = col_double(),
  complaint_date = col_date(format = ""),
  closed_date = col_date(format = ""),
  appointed_date = col_date(format = ""),
  resignation_date = col_date(format = ""),
  officer_birthyear = col_double()
)
ℹ Use `spec()` for the full column specifications.



── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────
cols(
  .default = col_character(),
  AllegationID = col_double(),
  OfficerID = col_double(),
  ShieldNo = col_double(),
  DaysOnForce = col_double(),
  ComplaintID = col_double(),
  IncidentDate = col_date(format = ""),
  ImpactedAge = col_double(),
  ReceivedDate = col_date(format = ""),
  CloseDate = col_date(format = "")
)
ℹ Use `spec()` for the full column specif

### Decile analysis: what does the distribution of complaints look like in each city?

#### Chicago

Create a dataframe, `chicago.grouped`, that groups complaints by officer ID (`link_UID`) and counts the number of complaints per officer

In [3]:
chicago.grouped <- chicago %>% group_by(link_UID) %>% summarise(n = n())

Create a dataframe, `chicago.grouped.deciles`, that divides the officers into ten equally-sized groups, each representing 10% of the officers based on their number of complaints. Then, calculate the number of complaints in each decile and the percent of total complaints that each decile's complaints represents.

In [4]:
chicago.grouped.deciles <-
chicago.grouped %>%
  arrange(desc(n)) %>%
  mutate(decile = ntile(n, 10)) %>%
  group_by(decile) %>%
  summarise(n = sum(n)) %>%
  mutate(perc = n/sum(n) * 100)

The top 10% of officers generating complaints (decile 10) accounted for 36.8% of the total complaints.

In [5]:
chicago.grouped.deciles

decile,n,perc
<int>,<int>,<dbl>
1,928,2.489938
2,928,2.489938
3,928,2.489938
4,1521,4.081030
5,1856,4.979877
6,2588,6.943923
7,3331,8.937483
8,4654,12.487255
9,6831,18.328414


This group of officers in the top decile had a median of 13 complaints.

In [6]:
chicago.grouped %>%
  arrange(desc(n)) %>%
  mutate(decile = ntile(n, 10)) %>%
  filter(decile == 10) %>%
  summarise(median = median(n))

median
<int>
13


#### NYC

Create a dataframe, `nyc.grouped`, that groups complaints by officer ID (`OfficerID`) and counts the number of complaints per officer. Note that for complaints that name a given officer multiple times, we only count the officer once. In other words, we make it so that a given police officer can have a maximum of one allegation under a given complaint.  

In [7]:
nyc.grouped <- nyc %>% 
               distinct(ComplaintID, OfficerID) %>% group_by(OfficerID) %>% 
               summarise(n = n())

Create a dataframe, `nyc.grouped.deciles`, that divides the officers into ten equally-sized groups, each representing 10% of the officers based on their number of complaints. Then, calculate the number of complaints in each decile and the percent of total complaints that each decile's complaints represents.

In [8]:
nyc.grouped.deciles <-
nyc.grouped %>%
  arrange(desc(n)) %>%
  mutate(decile = ntile(n, 10)) %>%
  group_by(decile) %>%
  summarise(n = sum(n)) %>%
  mutate(perc = n/sum(n) * 100)

The top 10% of officers generating complaints (decile 10) accounted for 31.7% of the total complaints.

In [9]:
nyc.grouped.deciles

decile,n,perc
<int>,<int>,<dbl>
1,2085,4.098522
2,2085,4.098522
3,2085,4.098522
4,2085,4.098522
5,2663,5.234707
6,4168,8.193112
7,4445,8.737616
8,6252,12.289668
9,8867,17.430020


This group of officers in the top decile had a median of 7 complaints.

In [10]:
nyc.grouped %>%
  arrange(desc(n)) %>%
  mutate(decile = ntile(n, 10)) %>%
  filter(decile == 10) %>%
  summarise(median = median(n))

median
<dbl>
7


#### Philly

Create a dataframe, `philly.grouped`, that groups complaints by officer ID (`officer_id`) and counts the number of complaints per officer. Note that for complaints that name a given officer multiple times, we only count the officer once. In other words, we make it so that a given police officer can have a maximum of one allegation under a given complaint.

In [11]:
philly.grouped <- philly %>%
  distinct(complaint_id, officer_id) %>% 
  group_by(officer_id) %>%
  summarise(n = n()) 

Create a dataframe, `philly.grouped.deciles`, that divides the officers into ten equally-sized groups, each representing 10% of the officers based on their number of complaints. Then, calculate the number of complaints in each decile and the percent of total complaints that each decile's complaints represents.

In [12]:
philly.grouped.deciles <-
philly.grouped %>%
  arrange(desc(n)) %>%
  mutate(decile = ntile(n, 10)) %>%
  group_by(decile) %>%
  summarise(n = sum(n)) %>%
  mutate(perc = n/sum(n) * 100)

The top 10% of officers generating complaints (decile 10) accounted for 28% of the total complaints.

In [13]:
philly.grouped.deciles

decile,n,perc
<int>,<int>,<dbl>
1,336,5.162083
2,336,5.162083
3,336,5.162083
4,336,5.162083
5,336,5.162083
6,540,8.296205
7,672,10.324167
8,737,11.322784
9,1056,16.223690


This group of officers in the top decile had a median of 5 complaints.

In [14]:
philly.grouped %>%
  arrange(desc(n)) %>%
  mutate(decile = ntile(n, 10)) %>%
  filter(decile == 10) %>%
  summarise(median = median(n))

median
<dbl>
5


### Disciplinary action analysis: what proportion of allegations result in discipline?

#### Chicago

In [15]:
table(chicago$final_finding) # only 530 sustained


   EX   NAF    NS    SU    UN 
 1408 19860  8822   490  4753 

In [16]:
chicago %>%
        group_by(final_outcome) %>% 
        summarise(n = n()) %>% 
        mutate(perc = n/sum(n) * 100) %>% 
        arrange(desc(perc)) %>% head()

final_outcome,n,perc
<chr>,<int>,<dbl>
No Action Taken,34172,91.6876845
Unknown,1873,5.0254897
NA,700,1.8781862
Reprimand,77,0.2066005
1 Day Suspension,60,0.1609874
Violation Noted,52,0.1395224


#### NYC

In [17]:
table(nyc$BoardCat)


Closed - Pending Litigation                  Exonerated 
                        175                       20498 
              Miscellaneous               Substantiated 
                       1000                        6692 
                  Truncated                   Unfounded 
                      27925                        6529 
            Unsubstantiated 
                      32871 

In [18]:
nyc %>% group_by(PenaltyDesc) %>% 
        summarise(n = n()) %>% 
        mutate(perc = n/sum(n) * 100) %>% 
        arrange(desc(perc)) %>% head()

PenaltyDesc,n,perc
<chr>,<int>,<dbl>
NA,89881,93.9293552
Instruction,1218,1.2728603
No penalty,1107,1.1568607
Formalized Training,913,0.9541227
Command Discipline - A,667,0.6970425
Command Discipline - B,656,0.6855471


#### Philly

In [19]:
philly %>% group_by(disciplinary_findings) %>% summarise(n = n()) %>% mutate(perc = n/sum(n) * 100) %>% arrange(desc(n))

disciplinary_findings,n,perc
<chr>,<int>,<dbl>
Not Applicable,7047,73.26887087
Pending,1474,15.32543148
Training/Counseling,855,8.88958203
Guilty Finding,188,1.95466833
No Guilty Findings,52,0.54065294
#VALUE!,2,0.02079434
